<img style="float: left; width: 220px;" src="images/shutterstock_318255209.jpg">

## <font color="grey"> $\quad$ Distinct proliferation rates - Figures </font>

$\newcommand{\vct}[1]{\mathbf{#1}}$
$\newcommand{\mtx}[1]{\mathbf{#1}}$
$\newcommand{\e}{\varepsilon}$
$\newcommand{\norm}[1]{\|#1\|}$
$\newcommand{\minimize}{\mathrm{minimize}\quad}$
$\newcommand{\maximize}{\mathrm{maximize}\quad}$
$\newcommand{\subjto}{\quad\text{subject to}\quad}$
$\newcommand{\R}{\mathbb{R}}$
$\newcommand{\trans}{T}$
$\newcommand{\ip}[2]{\langle {#1}, {#2} \rangle}$
$\newcommand{\zerovct}{\vct{0}}$
$\newcommand{\diff}[1]{\mathrm{d}{#1}}$
$\newcommand{\conv}{\operatorname{conv}}$
$\newcommand{\inter}{{\operatorname{int}}}$

### <font color="grey">  Table of Contents</font>

1. #### <a href='#chapter1'>Data preparation.</a>

2. #### <a href='#chapter2'>The molecular subtypes of melanoma present distinct ratios of clock-like mutations per unit of time.</a>

3. #### <a href='#chapter3'>Aging affects the intrinsic mutation rate of the molecular subtypes.</a>

4. #### <a href='#chapter4'>Signature 1 and signature 7 mutations are tightly correlated and together contribute to melanoma across all molecular subtypes.</a>

5. #### <a href='#chapter5'>The proportion and spectra of intrinsic and extrinsic mutations varies according to gender.</a>

In [ ]:
# Relevant R libraries
library(MASS)
library(robustbase)
library(robust)
library(ggplot2)
library(sfsmisc)
library(repr)
library(AER)
library(vcd)
library(zoo)
options(repr.plot.width=8, repr.plot.height=3)
library(gridExtra)
library(nnls)
library(stringr)
library(boot)
library(ggsci)
library(scales)

<a id='chapter1'></a>
###  <a id='chapter1'> <font color="grey">1. Data preparation </font></a>

In [ ]:
load('data/tcga_4class.Rdata')
data <- sigdata.tcga
cat("There are", dim(data)[1], "data samples\n")

In [ ]:
head(data)

### <font color="grey">  1.1 Data summary</font>

#### We first summarize how many samples are in each category.

In [ ]:
cat("TCGA data: \n")
cat("BRAF:", nrow(subset(data, Cohort=="BRAF")), "\n")
cat("NRAS:", nrow(subset(data, Cohort=="NRAS")), "\n")
cat("NF1:", nrow(subset(data, Cohort=="NF1")), "\n")
cat("Triple wild:", nrow(subset(data, Cohort=="W3")), "\n")
cat("Male BRAF:", nrow(subset(data, Cohort=="BRAF" & Gender=='MALE')), "\n")
cat("Male NRAS:", nrow(subset(data, Cohort=="NRAS" & Gender=='MALE')), "\n")
cat("Male NF1:", nrow(subset(data, Cohort=="NF1" & Gender=='MALE')), "\n")
cat("Male Triple wild:", nrow(subset(data, Cohort=="W3" & Gender=='MALE')), "\n")
cat("Female BRAF:", nrow(subset(data, Cohort=="BRAF" & Gender=='FEMALE')), "\n")
cat("Female NRAS:", nrow(subset(data, Cohort=="NRAS" & Gender=='FEMALE')), "\n")
cat("Female NF1:", nrow(subset(data, Cohort=="NF1" & Gender=='FEMALE')), "\n")
cat("Female Triple wild:", nrow(subset(data, Cohort=="W3" & Gender=='FEMALE')), "\n")
cat("MC1R information available for", nrow(subset(data, !is.na(MC1R))), "samples")

The **averages** datasets below contain mean and median information on the number of mutations by each age group. This is because sometimes we want to analyse the median mutation load for each age against age.

In [ ]:
averages <- read.csv('data/averages.csv', header=TRUE)
averages.brafnras <- read.csv('data/averages_brafnras.csv', header=TRUE)
averages.braf <- read.csv('data/averages_braf.csv', header=TRUE)
averages.nras <- read.csv('data/averages_nras.csv', header=TRUE)
averages.nf1 <- read.csv('data/averages_nf1.csv', header=TRUE)
averages.w3 <- read.csv('data/averages_w3.csv', header=TRUE)
averages.all <- read.csv('data/averages_all.csv', header=TRUE)
averages.male <- read.csv('data/averages_male.csv', header=TRUE)
averages.female <- read.csv('data/averages_female.csv', header=TRUE)
averages.gender <- read.csv('data/averages_gender.csv', header=TRUE)

#### Some overall statistics of the data
The conclusion from the below is that, on average, BRAF patients are younger than NRAS, which are much younger than NF1.

In [ ]:
w3 <- subset(data, Cohort=="W3")$Age
nras <- subset(data, Cohort=="NRAS")$Age
braf <- subset(data, Cohort=="BRAF")$Age
nf1 <- subset(data, Cohort=="NF1")$Age
cat("Mean age for BRAF:", mean(braf), "\n")
cat("Mean age for NRAS:", mean(nras), "\n")
cat("Mean age for NF1:", mean(nf1), "\n")
cat("Mean age for W3:", mean(w3), "\n")
cat("Mean age overall:", mean(data$Age), "\n")

Compute the average number of mutations (Signature 1 and total) for the whole dataset and for the subroups

In [ ]:
w3 <- subset(data, Cohort=="W3")
nras <- subset(data, Cohort=="NRAS")
braf <- subset(data, Cohort=="BRAF")
nf1 <- subset(data, Cohort=="NF1")
cat("Sig1 for BRAF:", mean(braf$Sig7Total), "\n")
cat("Sig1 for NRAS:", mean(nras$Sig7Total), "\n")
cat("Sig1 for NF1:", mean(nf1$Sig7Total), "\n")
cat("Sig1 for W3:", mean(w3$Sig7Total), "\n")
cat("Sig1 overall:", mean(data$Sig7Total), "\n")
cat("Total for BRAF:", mean(braf$TotalSNV), "\n")
cat("Total for NRAS:", mean(nras$TotalSNV), "\n")
cat("Total for NF1:", mean(nf1$TotalSNV), "\n")
cat("Total for W3:", mean(w3$TotalSNV), "\n")
cat("Total overall:", mean(data$TotalSNV), "\n")

These are just some auxiliary data sets that are used (or not) later throughout the analysis

In [ ]:
# Auxiliary columns that might come handy
data$Sig1Ratio <- data$Sig1Total/data$Age
data$Sig1pSig7 <- data$Sig1Rel+data$Sig7Rel
data$range <- cut(data$Age, c(0, 30, 40, 50, 60, 70, 80, 90), include.lowest=FALSE)
data$two.range <- cut(data$Age, c(0, 55, 90), include.lowest=FALSE)
data$four.range <- cut(data$Age, c(0, 30, 50, 70, 90), include.lowest=FALSE)
averages.all$two.range <- cut(averages.all$Age, c(0, 55, 90), include.lowest=FALSE)
mysig7range <- cut(data$Sig7Total, seq(0,4000,by=10), include.lowest=FALSE)
a <- sapply(mysig7range, as.character)
data$sig7range <- unname(sapply(gsub(".*\\((.+)\\,.*", "\\1", a), as.numeric))
mysig5range <- cut(data$Sig5Total, seq(0,4000,by=10), include.lowest=FALSE)
a <- sapply(mysig5range, as.character)
data$sig5range <- unname(sapply(gsub(".*\\((.+)\\,.*", "\\1", a), as.numeric))

#### MC1R Information
Introduce new data frame (name: **mydata**) for samples with MC1R information.

In [ ]:
# Add additional columns to the data
mydata <- subset(data, !is.na(MC1R))
mydata$Rallele <- as.factor(str_count(mydata$MC1R, 'R')>0)
levels(mydata$Rallele) <- c("0 R alleles", "1-2 R alleles")

In [ ]:
cat("TCGA data (1-2 MC1R alleles): \n")
cat("BRAF:", nrow(subset(mydata, Cohort=="BRAF" & Rallele == "1-2 R alleles")), "\n")
cat("NRAS:", nrow(subset(mydata, Cohort=="NRAS" & Rallele == "1-2 R alleles")), "\n")
cat("NF1:", nrow(subset(mydata, Cohort=="NF1" & Rallele == "1-2 R alleles")), "\n")
cat("Triple wild:", nrow(subset(mydata, Cohort=="W3" & Rallele == "1-2 R alleles")), "\n")

In [ ]:
cat("TCGA data (0 MC1R alleles): \n")
cat("BRAF:", nrow(subset(mydata, Cohort=="BRAF" & Rallele == "0 R alleles")), "\n")
cat("NRAS:", nrow(subset(mydata, Cohort=="NRAS" & Rallele == "0 R alleles")), "\n")
cat("NF1:", nrow(subset(mydata, Cohort=="NF1" & Rallele == "0 R alleles")), "\n")
cat("Triple wild:", nrow(subset(mydata, Cohort=="W3" & Rallele == "0 R alleles")), "\n")

In [ ]:
# For later use
sigs <- c("Sig1Med", "Sig7Med")
sigst <- c("Sig1Total", "Sig7Total")
cohorts <- c("BRAF", "NRAS", "NF1", "W3")
alleles <- c("0 R alleles", "1-2 R alleles")
genders <- c("MALE", "FEMALE")
avgsets <- list(averages.braf, averages.nras, averages.nf1, averages.w3)
cohortsets <- list(subset(data, Cohort=='BRAF'),subset(data, Cohort=='NRAS'),subset(data, Cohort=='NF1'),subset(data, Cohort=='W3'))
sig7ranges <- sort(unique(data$sig7range)[!is.na(unique(data$sig7range))])

###  <a id='chapter2'><font color="grey">  2. The molecular subtypes of melanoma present distinct ratios of clock-like mutations per unit of time</font></a>

Robust linear regression of Signature 1 against age, and subdivided by cohorts.

In [ ]:
plotA <- ggplot(averages, aes(Age, Sig1Med)) +  geom_smooth(method = "rlm", se=FALSE) + geom_point() +
    xlab("Age") + ylab("Signature 1") + theme_minimal()# + theme(legend.position="none")
plotB <- ggplot(subset(averages.all, Cohort=='BRAF' | Cohort=='NRAS'), aes(Age, Sig1Med, color=Cohort)) +  geom_smooth(method = "rlm", se=FALSE) + geom_point() +
    xlab("Age") + ylab("") +#scale_y_continuous(limits = c(0,2000)) +
    theme_minimal() + theme(legend.position="top")
grid.arrange(plotA, plotB, ncol=2)
p <- arrangeGrob(plotA, plotB, ncol=2)

In [ ]:
plot1 <- ggplot(averages, aes(Age, Sig1Med)) +  geom_smooth(method = "rlm", se=FALSE) + geom_point() +
    xlab("Age") + ylab("Signature 1") + theme_minimal()# + theme(legend.position="none")
plot2 <- ggplot(subset(averages.all, Cohort=='BRAF' | Cohort=='NRAS'), aes(Age, Sig1Med, color=Cohort)) +  geom_smooth(method = "rlm", se=FALSE) + geom_point() +
    xlab("Age") + ylab("") +#scale_y_continuous(limits = c(0,2000)) +
    theme_minimal() + theme(legend.position="top")
grid.arrange(plot1, plot2, ncol=2)
p <- arrangeGrob(plot1, plot2, ncol=2)

#### Robust linear regression coefficients with P-value

In [ ]:
res <- rlm(as.formula("Sig1Med ~ Age"), data=averages)
p.slope <- f.robftest(res, var = "Age")$p.value
p.intercept <- f.robftest(res, var = "(Intercept)")$p.value
cat("(Signature 1)", " Slope on robust regression:", unname(coef(res)['Age']), "with P-value", p.slope, "\n")
cat("(Signature 1)", " Intercept on robust regression:", unname(coef(res)['(Intercept)']), "with P-value", p.intercept, "\n\n")
for (i in seq(1:4)) {
    res <- rlm(as.formula("Sig1Med ~ Age"), data=avgsets[[i]])
    p.slope <- f.robftest(res, var = "Age")$p.value
    p.intercept <- f.robftest(res, var = "(Intercept)")$p.value
    cat("(Signature 1", cohorts[i],")", " Slope on robust regression:", unname(coef(res)['Age']), "with P-value", p.slope, "\n")
    cat("(Signature 1", cohorts[i],")", " Intercept on robust regression:", unname(coef(res)['(Intercept)']), "with P-value", p.intercept, "\n\n")
}

#### Spearman rho

In [ ]:
# Compute confidence intervals by bootstrapping
spearman.rho.sig1med.age <- function(data, indices) {
  d <- data[indices,] # allows boot to select sample 
  fit <- cor.test(d$Age, d$Sig1Med, method="spearman", alternative="greater", data=d, exact=FALSE)
  return(fit$estimate)
} 

In [ ]:
correlation <- cor.test(averages$Age, averages$Sig1Med, method="spearman", alternative="greater", exact=FALSE)
cat("Spearman correlation with age on complete dataset:", correlation$estimate, "with P-value", correlation$p.value, "\n\n")
for (i in seq(1:4)) {
    correlation <- cor.test(avgsets[[i]]$Age, avgsets[[i]]$Sig1Med, method="spearman", alternative="greater", exact=FALSE)
    cat("Spearman rho correlation with age on", cohorts[i], "dataset:", correlation$estimate, "with P-value", correlation$p.value, "\n")
    }

#### Median Signature 1/age by cohorts

In [ ]:
plotC <- ggplot(data, aes(Cohort, (Sig1Total)/Age, fill=Cohort)) +
    geom_boxplot() + ylab("Signatures 1 mutations / year") + theme_minimal() + guides(fill=FALSE) + coord_cartesian(ylim = c(0,3))
plotF <- ggplot(data, aes(Cohort, (Sig7Total)/Age, fill=Cohort)) +
    geom_boxplot() + ylab("Signatures 7 mutations / year") + theme_minimal() + guides(fill=FALSE) + coord_cartesian(ylim = c(0,60))
grid.arrange(plotC, plotF, ncol=2)

In [ ]:
#ggsave("./figures/Figure1EF.pdf", p, scale=1, width=8, height=4)

In [ ]:
md.sig1sig7 <- function(data, indices) {
    mydata <- data[indices,]
    return(median((mydata$Sig1Total)/mydata$Age))
}

In [ ]:
for (i in seq(1:4)) {
    res <- md.sig1sig7(subset(data, Cohort==cohorts[i]),)
    cat("Median ratio for", cohorts[i], ": ", res, "\n\n")
}

#### Mann-Whitney U test with Bonferroni correction
Test whether the ratios Sig1/Age differ in median across the subtypes. Using Sig1+Sig7, otherwise not significant difference between BRAF and NRAS.

In [ ]:
# Do a two-sided Wilcoxon test
newdata <- data
pwres <- pairwise.wilcox.test((newdata$Sig1Total)/newdata$Age, newdata$Cohort, exact=FALSE, paired=FALSE, alternative="two.sided", p.adj="bonferroni")
pwres$p.value

In [ ]:
# Only compare BRAF and NRAS
md.sig1 <- function(data, indices) {
    mydata <- data[indices,]
    return(median(mydata$Sig1Total/mydata$Age))
}

In [ ]:
newdata.braf <- subset(data, Cohort=='BRAF')
newdata.nras <- subset(data, Cohort=='NRAS')
pwres <- wilcox.test(newdata.braf$Sig1Total/newdata.braf$Age, newdata.nras$Sig1Total/newdata.nras$Age, alternative="two.sided", exact=FALSE, pairwise=FALSE)
cat("P-value for BRAF vs NRAS:", pwres$p.value, '\n')

#### Test the difference in the presence of 0 or 1-2 MC1R R alleles

In [ ]:
p <- ggplot(transform(mydata,
      Cohort=factor(Cohort,levels=c("BRAF","NRAS","NF1","W3"))), aes(Rallele, Sig1Total/Age, fill=Rallele)) +
    geom_boxplot() + ylab("Signatures 1/ Age") + theme_minimal() + #coord_cartesian(ylim = c(0,55)) + 
    guides(fill=FALSE) + facet_wrap(~ Cohort, ncol=4) + xlab("MC1R")
grid.arrange(p, ncol=1)

In [ ]:
#ggsave("./figures/SupFigure1.pdf", p, scale=1, width=8, height=4)

#### Wilcoxon rank-sum test to see whether the ratios Sig1/Age differ depending on MC1R status

In [ ]:
newdataR <- subset(mydata, Rallele=='1-2 R alleles')
newdata0 <- subset(mydata, Rallele=='0 R alleles')
medR <- median(newdataR$Sig1Total/newdataR$Age)
med0 <- median(newdata0$Sig1Total/newdata0$Age)
cat("Median ratio: ", medR/med0, "\n\n")
pwres <- wilcox.test(newdataR$Sig1Total/newdataR$Age, newdata0$Sig1Total/newdata0$Age, paired=FALSE, exact=FALSE)
cat("P-value:", pwres$p.value, '\n')

#### Confidence interval via bootstrapping

In [ ]:
md <- function(data, indices) {
    tempdata <- data[indices,]
    return(median(subset(tempdata, Rallele=='1-2 R alleles')$Sig1Ratio/median(subset(tempdata, Rallele=='0 R alleles')$Sig1Ratio)))
}
bootmed.all <- boot(data=mydata, statistic=md, R=10000)
boot.ci(bootmed.all)

In [ ]:
for (i in seq(1:4)) {
    newdataR <- subset(mydata, Cohort==cohorts[i] & Rallele=='1-2 R alleles')
    newdata0 <- subset(mydata, Cohort==cohorts[i] & Rallele=='0 R alleles')
    medR <- median(newdataR$Sig1Total/newdataR$Age)
    med0 <- median(newdata0$Sig1Total/newdata0$Age)
    cat("Median ratio: ", medR/med0, "\n")
    pwres <- wilcox.test(newdataR$Sig1Total/newdataR$Age, newdata0$Sig1Total/newdata0$Age, paired=FALSE, exact=FALSE)
    cat("P-value for", cohorts[i], ":", pwres$p.value, '\n')
}

#### Repeat with Signature 7

In [ ]:
plotD <- ggplot(averages, aes(Age, Sig7Med)) +  geom_smooth(method = "rlm", se=FALSE) + geom_point() +
    xlab("Age") + ylab("Signature 7") + theme_minimal()# + theme(legend.position="none")
plotE <- ggplot(subset(averages.all, Cohort=='BRAF' | Cohort=='NRAS'), aes(Age, Sig7Med, color=Cohort)) +  geom_smooth(method = "rlm", se=FALSE) + geom_point() +
    xlab("Age") + ylab("") +#scale_y_continuous(limits = c(0,2000)) +
    theme_minimal() + theme(legend.position="top")
grid.arrange(plotD, plotE, ncol=2)
#p <- arrangeGrob(plot1, plot2, ncol=2)

In [ ]:
#ggsave("./figures/Figure2AB.pdf", p, scale=1, width=8, height=3)

In [ ]:
correlation <- cor.test(averages$Age, averages$Sig7Med, method="spearman", alternative="greater", exact=FALSE)
cat("Spearman correlation with age on complete dataset:", correlation$estimate, "with P-value", correlation$p.value, "\n\n")
for (i in seq(1:4)) {
    correlation <- cor.test(avgsets[[i]]$Age, avgsets[[i]]$Sig7Med, method="spearman", alternative="greater", exact=FALSE)
    cat("Spearman rho correlation with age on", cohorts[i], "dataset:", correlation$estimate, "with P-value", correlation$p.value, "\n")
    }

In [ ]:
plot1 <- ggplot(data, aes(Cohort, (Sig7Total)/Age, fill=Cohort)) +
    geom_boxplot() + ylab("Signatures 7 / year") + theme_minimal() + guides(fill=FALSE) #+ coord_cartesian(ylim = c(0,3))
grid.arrange(plot1, ncol=1)
p <- arrangeGrob(plot1, ncol=1)

In [ ]:
#ggsave("./figures/Figure2C.pdf", p, scale=1, width=8, height=4)

In [ ]:
# Do a two-sided Wilcoxon test
newdata <- data
pwres <- pairwise.wilcox.test((newdata$Sig7Total)/newdata$Age, newdata$Cohort, exact=FALSE, paired=FALSE, alternative="two.sided", p.adj="bonferroni")
pwres$p.value

###  <a id='chapter3'><font color="grey">  3. Aging affects the intrinsic mutation rate of the molecular subtypes.</font></a>

In [ ]:
plot2B <- ggplot(subset(data, range != "(0,30]" & range != "(40,50]" & range != "(30,40]"), aes(x=Sig1Total, fill=range)) +
    geom_density(alpha=0.7) + theme_minimal() + xlab("Mutations") + ylab("Frequency")
plot2A <- ggplot(averages.gender, aes(Age, Sig1Med/Age)) +  geom_smooth(method = "rlm", se=FALSE) + geom_point() +
    xlab("Age") + ylab("Signature 1 / Age") + theme_minimal() + theme(legend.position="none") + coord_cartesian(ylim = c(0,3))
grid.arrange(plot2A, plot2B, ncol=2)
#p <- arrangeGrob(plot1, plot2, ncol=2)

In [ ]:
#ggsave("./figures/Figure4AB.pdf", plot=p, scale=1, width=8, height=4)

#### Spearman rho of Sig1/Age against age

In [ ]:
sig1ageagecor <- cor.test(averages$Sig1Med/averages$Age, averages$Age, method="spearman", alternative="two.sided", exact=FALSE)
cat("Spearman correlation between Sig1/Age and Age:", sig1ageagecor$estimate, "with P-value", sig1ageagecor$p.value, "\n")

#### Test Spearman rho correlation across subtypes

In [ ]:
sig1ageagecor.braf <- cor.test(averages.braf$Sig1Med/averages.braf$Age, averages.braf$Age, method="spearman", alternative="two.sided", exact=FALSE)
cat("Spearman correlation between Sig1/Age and Age for BRAF:", sig1ageagecor.braf$estimate, "with P-value", sig1ageagecor.braf$p.value, "\n")
sig1ageagecor.nras <- cor.test(averages.nras$Sig1Med/averages.nras$Age, averages.nras$Age, method="spearman", alternative="two.sided", exact=FALSE)
cat("Spearman correlation between Sig1/Age and Age for NRAS:", sig1ageagecor.nras$estimate, "with P-value", sig1ageagecor.nras$p.value, "\n")
sig1ageagecor.w3 <- cor.test(averages.w3$Sig1Med/averages.w3$Age, averages.w3$Age, method="spearman", alternative="two.sided", exact=FALSE)
cat("Spearman correlation between Sig1/Age and Age for NRAS:", sig1ageagecor.w3$estimate, "with P-value", sig1ageagecor.w3$p.value, "\n")

### <font color="grey">  Intermezzo: the model.</font>

In order to more accurately quantify the accumulation of mutations over time, we need to make some model assumptions. While these assumptions may represent oversimplifications, they lead to systematic biases that would hold across different cohorts, and still allow to compare properties of the different types of samples to each other.

For the accumulation of mutations at time $t$, [Podolskiy et al](http://www.nature.com/articles/ncomms12157) postulate the following model, assumed to be valid for a certain age range. At each age $t$, the number of muations $N(t)$ is approximately distributed according to a Poisson distribution with rate $\lambda(t)$,

\begin{equation*}
  \mtx{P}\{N(t) = n\} = \frac{\lambda(t)^n e^{-\lambda(t)}}{n!}.
\end{equation*}

While [Podolskiy et al](http://www.nature.com/articles/ncomms12157) consider the total number of mutations and argue statistically via the law of large numbers, it is reasonable to assume that the accumulation of units of signature 1 also follows a Poisson-like distribution, or a mixture of Poisson distributions with one dominant component. This component is assumed to describe the accumulation of mutations before clonal expansion, after which the dynamics change. 

Suppose that at age $t$ we have $N_1(t),\dots,N_s(t)$ samples. We estimate the parameter $\lambda(t) = \lambda(t,X_1,\dots,X_p) = \mtx{E}[N(t) \ | \ X_1,\dots,X_p]$ using Poisson regression,

\begin{equation*}
  \log \mtx{E}[N(t) \ | \ X_1,\dots,X_p ] = \beta_0 + \sum_{i=1}^p \beta_i X_i,
\end{equation*}

where the $\beta_i$ are allowed to depend on $t$. In [Podolskiy et al](http://www.nature.com/articles/ncomms12157), the authors look at small time intervals and determine the Poisson peaks for each of these intervals. 
The practical problem with the approach described above is that we do not have enough data for reliably estimating Poisson average at each time. We can still try to use, for eath $t$ in a suitable range and $\Delta t$ (for example, 5 years), to use the interval $(t-\Delta t, t+\Delta t]$. A different approach would be to estimate the parameters for all times simultaneously.

Under simplifying assumptions, we can just include age (time) $t$ among the dependent variables and solve a standard Poisson model,

\begin{equation*}
  \log \mtx{E}[N \ | \ t, X_1,\dots,X_p ] = \beta_0 + \alpha t + \sum_{i=1}^p \beta_i X_i,
\end{equation*}

We first try to incorporate as many covariates as possible in the model.

In [ ]:
plot2C <- ggplot(averages, aes(Age, Sig1Mean)) +  geom_point() +
    geom_smooth(method = "glm", aes(Age, as.integer(Sig1Mean)), formula = y ~ x, data=averages, se=FALSE, method.args = (family="poisson")) +
    xlab("Age") + ylab("Signature 1") + theme_minimal() + coord_cartesian(ylim = c(0,200))# + theme(legend.position="none")
plot2D <- ggplot(averages.all, aes(Age, Sig1Mean, color=Cohort)) +  
    geom_smooth(method = "glm", aes(Age, as.integer(Sig1Mean)), formula = y ~ x, data=averages.all, se=FALSE, method.args = (family="poisson")) + 
    geom_point() + xlab("Age") + ylab("") + theme_minimal() + theme(legend.position="top") + coord_cartesian(ylim = c(0,200))
#p <- arrangeGrob(plot2C, plot2D, ncol=2)
grid.arrange(plot2C, plot2D, ncol=2)

In [ ]:
#ggsave("./figures/Figure4CD.pdf", plot=p, scale=1, width=8, height=3)

In the model above, we use the formula $N(t)=N_0e^{\alpha t}$ to model the accumulation of Signature 1 mutations. We estimate the parameters $\alpha$ and $N_0$ of this model using negative binomial regression. A standard algebraic computation shows that the we should expect a decrease in the ratio of mutations $N/t$ for ages $t<1/\alpha$ and an increase for ages $t>1/\alpha$. We compute the critical age threshold below.

In [ ]:
res <- glm.nb(as.integer(Sig1Med) ~ Age, data=averages)
alpha <- coef(res)["Age"]
cat("alpha: ", alpha, "Inverse of alpha: ", 1/alpha, "\n\n")
res.braf <- glm.nb(as.integer(Sig1Med) ~ Age, data=averages.braf)
alpha.braf <- coef(res.braf)["Age"]
cat("alpha for BRAF: ", alpha.braf, "Inverse of alpha: ", 1/alpha.braf, "\n")
res.nras <- glm.nb(as.integer(Sig1Med) ~ Age, data=averages.nras)
alpha.nras <- coef(res.nras)["Age"]
cat("alpha for NRAS: ", alpha.nras, "Inverse of alpha: ", 1/alpha.nras, "\n")
res.w3 <- glm.nb(as.integer(Sig1Med) ~ Age, data=averages.w3)
alpha.w3 <- coef(res.w3)["Age"]
cat("alpha for W3: ", alpha.w3, "Inverse of alpha: ", 1/alpha.w3)

Get confidence intervals and P values

In [ ]:
expalpha <- function(data, indices) {
  d <- data[indices,] # allows boot to select sample 
  res <- glm.nb(as.integer(d$Sig1Total) ~ d$Age, data = d)
  return(1/unname(exp(coef(res)["d$Age"])))
} 
results <- boot(data=data, statistic=alpha, R=1000)
boot.ci(results, type="basic")
results.braf <- boot(data=subset(data, Cohort=='BRAF'), statistic=alpha, R=1000)
boot.ci(results.braf, type="basic")
results.nras <- boot(data=subset(data, Cohort=='NRAS'), statistic=alpha, R=1000)
boot.ci(results.nras, type="basic")
results.w3 <- boot(data=subset(data, Cohort=='W3'), statistic=alpha, R=1000)
boot.ci(results.w3, type="basic")

We next fit the curves $N_0e^{\alpha t}$ with the estimated parameters to the data.

In [ ]:
expfun.braf <- function(x) {
    return(unname(exp(coef(res.braf)["(Intercept)"]))*exp(unname(coef(res.braf)["Age"])*x)/x)
    }
expfun.nras <- function(x) {
    return(exp(unname(coef(res.nras)["(Intercept)"]))*exp(unname(coef(res.nras)["Age"])*x)/x)
    }
expfun.w3 <- function(x) {
    return(exp(unname(coef(res.w3)["(Intercept)"]))*exp(unname(coef(res.w3)["Age"])*x)/x)
    }

In [ ]:
line.braf <- data.frame(x = seq(20,90, by=0.1), y=sapply(seq(20,90, by=0.1), expfun.braf))
line.nras <- data.frame(x = seq(30,90, by=0.1), y=sapply(seq(30,90, by=0.1), expfun.nras))
line.w3 <- data.frame(x = seq(30,90, by=0.1), y=sapply(seq(30,90, by=0.1), expfun.w3))

In [ ]:
plot2E1 <- ggplot(averages.braf, aes(Age, Sig1Med/Age)) + geom_point() +
    xlab("Age") + ylab("Signature 1 / Age") + theme_minimal() + theme(legend.position="none") + ggtitle('BRAF') + coord_cartesian(ylim = c(0,3))+ geom_line(data=line.braf, aes(x, y), color="blue", size=1)
plot2E2 <- ggplot(averages.nras, aes(Age, Sig1Mean/Age)) + geom_point() +
    xlab("Age") + ylab("") + theme_minimal() + theme(legend.position="none") + ggtitle('NRAS') + coord_cartesian(ylim = c(0,3))+ geom_line(data=line.nras, aes(x, y), color="blue", size=1)
plot2E3 <- ggplot(averages.w3, aes(Age, Sig1Mean/Age)) + geom_point() +
    xlab("Age") + ylab("") + theme_minimal() + theme(legend.position="none") + ggtitle('W3') + coord_cartesian(ylim = c(0,3))+ geom_line(data=line.w3, aes(x, y), color="blue", size=1)
grid.arrange(plot2E1, plot2E2, plot2E3, ncol=3, nrow=1)
plot2E <- arrangeGrob(plot1, plot2, plot3, ncol=3, nrow=1)

In [ ]:
#ggsave("./figures/Figure2E.pdf", plot=p, scale=1, width=8, height=3)

**Todo** P-values and confidence intervals

###  <a id='chapter4'><font color="grey">  4. Signature 1 and signature 7 mutations are tightly correlated and together contribute to melanoma across all molecular subtypes.  </font></a>

In [ ]:
# Compute averages of relative contribution of sig 1 and sig 7
nosig.braf <- 1-mean(subset(data, Cohort=='BRAF')$Sig1pSig7)
nosig.nras <- 1-mean(subset(data, Cohort=='NRAS')$Sig1pSig7)
nosig.nf1 <- 1-mean(subset(data, Cohort=='NF1')$Sig1pSig7)
nosig.w3 <- 1-mean(subset(data, Cohort=='W3')$Sig1pSig7)
sig7.braf <- mean(subset(data, Cohort=='BRAF')$Sig7Rel)
sig7.nras <- mean(subset(data, Cohort=='NRAS')$Sig7Rel)
sig7.nf1 <- mean(subset(data, Cohort=='NF1')$Sig7Rel)
sig7.w3 <- mean(subset(data, Cohort=='W3')$Sig7Rel)
sig1.braf <- mean(subset(data, Cohort=='BRAF')$Sig1Rel)
sig1.nras <- mean(subset(data, Cohort=='NRAS')$Sig1Rel)
sig1.nf1 <- mean(subset(data, Cohort=='NF1')$Sig1Rel)
sig1.w3 <- mean(subset(data, Cohort=='W3')$Sig1Rel) 
sig5.braf <- mean(subset(data, Cohort=='BRAF')$Sig5Rel)
sig5.nras <- mean(subset(data, Cohort=='NRAS')$Sig5Rel)
sig5.nf1 <- mean(subset(data, Cohort=='NF1')$Sig5Rel)
sig5.w3 <- mean(subset(data, Cohort=='W3')$Sig5Rel) 

In [ ]:
proportions <- c(sig1.braf, sig1.nras, sig1.nf1, sig1.w3, 
                 sig7.braf, sig7.nras, sig7.nf1, sig7.w3, 
                 sig5.braf, sig5.nras, sig5.nf1, sig5.w3,
                 nosig.braf, nosig.nras, nosig.nf1, nosig.w3)
labels <- c(rep(c('BRAF','NRAS','NF1','W3'),4))
types <- c(rep("Sig1",4), rep("Sig7",4), rep("Sig5",4), rep("Nosig",4))
newdata <- data.frame(labels, proportions, types)
plot3A <- ggplot(data=newdata[order(newdata$types),], aes(labels, proportions, fill=types)) + geom_bar(stat = "identity") + 
    xlab("Subtypes") + ylab("Mean proportion of Signatures") + theme_minimal() + 
    scale_fill_discrete(name="Mutation signature", breaks=c("Sig1","Sig7","Sig5","Nosig"), labels=c("Signature 1", "Signature 7", "Signature 5", "Other"))
grid.arrange(plot3A)

In [ ]:
#ggsave("./figures/Figure3A.pdf", plot=p, scale=1, width=8, height=3)

Plot Signature 1 against Signature 7

In [ ]:
# Compute median sig1 for each sig7
averages.sig7 <- data.frame(Sig7Total=sig7ranges)
averages.sig7$Sig1Med <- NA
for (s in sig7ranges) {
  i <- match(s,averages.sig7$Sig7Total)
  averages.sig7[i,c("Sig1Med")] <- median(data[which(data$sig7range==s),"Sig1Total"])
}
# Compute median sig1 for each sig7 subdivided by cohorts
averages.sig7.cohort <- data.frame(Sig7Total=integer(), Sig1Med=double(), Cohort=character())
for (j in seq(4)) {
    sig7 <- sort(unique(subset(data,Cohort==cohorts[j])$sig7range)[!is.na(unique(subset(data,Cohort==cohorts[j])$sig7range))])
    averages.temp <- data.frame(Sig7Total=sig7, Sig1Med=as.double(NA))
    for (s in sig7) {
        i <- match(s,averages.temp$Sig7Total)
        averages.temp[i,c("Sig1Med")] <- median(data[which(data$sig7range==s & data$Cohort==cohorts[j]),"Sig1Total"])
        }
    averages.temp$Cohort <- cohorts[j]
    averages.sig7.cohort <- rbind(averages.sig7.cohort, averages.temp)
    }

In [ ]:
plot3B <- ggplot(subset(averages.sig7, Sig7Total<2500), aes(Sig7Total, Sig1Med)) +  geom_smooth(method = "rlm", se=FALSE) + geom_point() +
    xlab("Signature 7") + ylab("Signature 1") + theme_minimal()# + theme(legend.position="none")
plot3C <- ggplot(subset(averages.sig7.cohort, (Cohort=='BRAF' | Cohort=='NRAS') & Sig7Total<2500), aes(Sig7Total, Sig1Med, color=Cohort)) +  geom_smooth(method = "rlm", se=FALSE) + geom_point() +
    xlab("Signature 7") + ylab("") +# scale_y_continuous(limits = c(0,2000)) +
    theme_minimal() + theme(legend.position="top")
grid.arrange(plot3B, plot3C, ncol=2)
#p <- arrangeGrob(plot1, plot2, ncol=2)

In [ ]:
#ggsave("./figures/Figure3BC.pdf", plot=p, scale=1, width=8, height=4)

In [ ]:
# Compute median sig1 for each sig5
averages.sig5 <- data.frame(Sig5Total=sig5ranges)
averages.sig5$Sig1Med <- NA
for (s in sig5ranges) {
  i <- match(s,averages.sig5$Sig5Total)
  averages.sig5[i,c("Sig1Med")] <- median(data[which(data$sig5range==s),"Sig1Total"])
}
# Compute median sig1 for each sig7 subdivided by cohorts
averages.sig5.cohort <- data.frame(Sig5Total=integer(), Sig1Med=double(), Cohort=character())
for (j in seq(4)) {
    sig5 <- sort(unique(subset(data,Cohort==cohorts[j])$sig5range)[!is.na(unique(subset(data,Cohort==cohorts[j])$sig5range))])
    averages.temp <- data.frame(Sig5Total=sig5, Sig1Med=as.double(NA))
    for (s in sig5) {
        i <- match(s,averages.temp$Sig5Total)
        averages.temp[i,c("Sig1Med")] <- median(data[which(data$sig5range==s & data$Cohort==cohorts[j]),"Sig1Total"])
        }
    averages.temp$Cohort <- cohorts[j]
    averages.sig5.cohort <- rbind(averages.sig5.cohort, averages.temp)
    }

#### Robust regression

In [ ]:
res<-rlm(Sig1Med ~ Sig7Total, data=averages.sig7)
slope.total <- unname(coef(res)['Sig7Total'])
p1.total <- f.robftest(res, var = "Sig7Total")$p.value
cat("(Total) Slope on robust regression:", slope.total, "with P-value", p1.total, "\n")

#### Spearman rho correlation

In [ ]:
sig1sig7cor <- cor.test(averages.sig7$Sig7Total, averages.sig7$Sig1Med, method="spearman", alternative="two.sided", exact=FALSE)
cat("Spearman correlation between Signature 1 and Signature 7:", sig1sig7cor$estimate, "with P-value", sig1sig7cor$p.value, "\n")

#### Robust regression across subtypes

In [ ]:
for (i in seq(1:4)) {
    res<-rlm(Sig1Med ~ Sig7Total, data=subset(averages.sig7.cohort, Cohort==cohorts[i]))
    slope.total <- unname(coef(res)['Sig7Total'])
    p1.total <- f.robftest(res, var = "Sig7Total")$p.value
    cat("(Total) Slope on robust regression for", cohorts[i], ":", slope.total, "with P-value", p1.total, "\n")
    }

#### Confidence intervals

In [ ]:
ratio <- function(data, indices) {
  d <- data[indices,] # allows boot to select sample 
  res <- rlm(d$Sig1Med ~ d$Sig7Total, data = d)
  return(unname(coef(res)["d$Sig7Total"]))
} 
results <- boot(data=subset(averages.sig7.cohort, Cohort=='BRAF' & Sig7Total>0), statistic=ratio, R=10000)
boot.ci(results, type="bca")

#### Spearman rho across subtypes

In [ ]:
# Determine slope and intersept with corresponding P values for robust linear regression on all the data
cat("Spearman rho correlation between Signature 1 and Signature 7\n\n")
for (i in seq(1:4)) {
    sig1sig7cor <- cor.test(subset(averages.sig7.cohort, Cohort==cohorts[i])$Sig7Total, subset(averages.sig7.cohort, Cohort==cohorts[i])$Sig1Med, method="spearman", alternative="two.sided", exact=FALSE)
    cat("(", cohorts[i], ")", "Spearman correlation between Signature 1 and Signature 7:", sig1sig7cor$estimate, "with P-value", sig1sig7cor$p.value, "\n")
}

#### Mann-Whitney U test for ratio Sig1/Sig7 across subtypes

In [ ]:
# Do a two-sided Wilcoxon test
newdata <- subset(averages.sig7.cohort, Sig7Total>0)
pwres <- pairwise.wilcox.test(newdata$Sig1Med/newdata$Sig7Total, newdata$Cohort, exact=FALSE, p.adj="bonferroni")
pwres$p.value

#### Wilcoxon rank-sum test for ratio Sig1/Sig7 for BRAF and NRAS

In [ ]:
# Only compare BRAF and NRAS
newdata.braf <- subset(data, Cohort=='BRAF' & Sig7Total>0)
newdata.nras <- subset(data, Cohort=='NRAS' & Sig7Total>0)
pwres <- wilcox.test(newdata.braf$Sig1Total/newdata.braf$Sig7Total, newdata.nras$Sig1Total/newdata.nras$Sig7Total, exact=FALSE, pairwise=FALSE)
cat("P-value for BRAF vs NRAS:", pwres$p.value, '\n')

#### Effect of MC1R

In [ ]:
# Determine slope and intersept with corresponding P values for robust linear regression on all the data
cat("Spearman rho correlation between Signature 1 and Signature 7\n\n")
for (i in seq(1:2)) {
    sig1sig7cor <- cor.test(subset(mydata, Rallele==alleles[i])$Sig7Total, subset(mydata, Rallele==alleles[i])$Sig1Total, method="spearman", alternative="two.sided", exact=FALSE)
    cat("(", alleles[i], ")", "Spearman correlation between Signature 1 and Signature 7:", sig1sig7cor$estimate, "with P-value", sig1sig7cor$p.value, "\n")
}

#### Effect of Gender

In [ ]:
# Compute averages in separate data frame (MALE)
sig7male <- sort(unique(subset(data,Gender=="MALE")$sig7range)[!is.na(unique(subset(data,Gender=="MALE")$sig7range))])
averages.male.sig7 <- data.frame(Sig7Total=sig7male)
averages.male.sig7$Sig1Med <- NA
for (s in sig7male) {
  i <- match(s,averages.male.sig7$Sig7Total)
  averages.male.sig7[i,c("Sig1Med")] <- median(data[which(data$sig7range==s & data$Gender=="MALE"),"Sig1Total"])
}
# Compute averages in separate data frame (FEMALE)
sig7female <- sort(unique(subset(data,Gender=="FEMALE")$sig7range)[!is.na(unique(subset(data,Gender=="FEMALE")$sig7range))])
averages.female.sig7 <- data.frame(Sig7Total=sig7female)
averages.female.sig7$Sig1Med <- NA
for (s in sig7female) {
  i <- match(s,averages.female.sig7$Sig7Total)
  averages.female.sig7[i,c("Sig1Med")] <- median(data[which(data$sig7range==s & data$Gender=="FEMALE"),"Sig1Total"])
}
averages.male.sig7$Gender <- "MALE"
averages.female.sig7$Gender <- "FEMALE"
averages.gender.sig7 <- rbind(averages.male.sig7, averages.female.sig7)

In [ ]:
plot1 <- ggplot(subset(averages.sig7, Sig7Total<2500), aes(Sig7Total, Sig1Med)) +  geom_smooth(method = "lm", se=FALSE) + geom_point() +
    xlab("Signature 7") + ylab("Signature 1") + theme_minimal()# + theme(legend.position="none")
plot2 <- ggplot(subset(averages.gender.sig7, Sig7Total<2500), aes(Sig7Total, Sig1Med, color=Gender)) +  geom_smooth(method = "lm", se=FALSE) + geom_point() +
    xlab("Signature 7") + ylab("Signature 1") +# scale_y_continuous(limits = c(0,2000)) +
    theme_minimal() + theme(legend.position="top")
grid.arrange(plot1, plot2, ncol=2)
p <- arrangeGrob(plot1, plot2, ncol=2)

In [ ]:
# Determine slope and intersept with corresponding P values for robust linear regression on all the data
cat("Spearman rho correlation between Signature 1 and Signature 7\n\n")
for (i in seq(1:2)) {
    sig1sig7cor <- cor.test(subset(data, Gender==genders[i])$Sig7Total, subset(data, Gender==genders[i])$Sig1Total, method="spearman", alternative="two.sided", exact=FALSE)
    cat("(", genders[i], ")", "Spearman correlation between Signature 1 and Signature 7:", sig1sig7cor$estimate, "with P-value", sig1sig7cor$p.value, "\n")
}

###  <a id='chapter5'><font color="grey">  5. The proportion and spectra of intrinsic and extrinsic mutations varies according to gender.</font></a>

There is not a significant difference between the ratio of Signature 1 mutations and gender overall, and it is hard to really detect any differences.

In [ ]:
# Compute averages in separate data frame (MALE)
agemale <- sort(unique(subset(data,Gender=="MALE")$Age)[!is.na(unique(subset(data,Gender=="MALE")$Age))])
averages.male.age <- data.frame(Age=agemale)
averages.male.age$Sig1Med <- NA
for (s in agemale) {
  i <- match(s,averages.male.age$Age)
  averages.male.age[i,c("Sig1Med")] <- median(data[which(data$agemale==s & data$Gender=="MALE"),"Sig1Total"])
  averages.male.age[i,c("Sig7Med")] <- median(data[which(data$agemale==s & data$Gender=="MALE"),"Sig7Total"])
}
# Compute averages in separate data frame (FEMALE)
agefemale <- sort(unique(subset(data,Gender=="FEMALE")$Age)[!is.na(unique(subset(data,Gender=="FEMALE")$Age))])
averages.female.age <- data.frame(Age=agefemale)
averages.female.age$Sig1Med <- NA
for (s in agefemale) {
  i <- match(s,averages.female.age$Age)
  averages.female.age[i,c("Sig1Med")] <- median(data[which(data$agefemale==s & data$Gender=="FEMALE"),"Sig1Total"])
  averages.female.age[i,c("Sig7Med")] <- median(data[which(data$agefemale==s & data$Gender=="FEMALE"),"Sig7Total"])
}
averages.male.age$Gender <- "MALE"
averages.female.age$Gender <- "FEMALE"
averages.gender.age <- rbind(averages.male.age, averages.female.age)

In [ ]:
plot4A <- ggplot(data, aes(Gender, Sig1Total/Age, fill=Gender)) +
    geom_boxplot() + ylab("Signatures 1/ Age") + theme_minimal() + coord_cartesian(ylim = c(0,2)) + 
    scale_fill_aaas() +
    guides(fill=FALSE) + xlab("")
plot4B <- ggplot(transform(subset(data, Cohort=='BRAF' | Cohort=='NRAS'),
      Cohort=factor(Cohort,levels=c("BRAF","NRAS"))), aes(Gender, Sig1Total/Age, fill=Gender)) +
    scale_fill_aaas() +
    geom_boxplot() + ylab("Signatures 1/ Age") + theme_minimal() + #coord_cartesian(ylim = c(0,55)) + 
    guides(fill=FALSE) + facet_wrap(~ Cohort, ncol=4) + xlab("")
grid.arrange(plot4A, plot4B, ncol=2)
#p <- arrangeGrob(plot1, plot2, ncol=2)

In [ ]:
#ggsave("./figures/Figure4AB.pdf", plot=p, scale=1, width=8, height=4)

#### Spearman rho correlation and ratio test

In [ ]:
correlation <- cor.test(averages$Age, averages$Sig1Med, method="spearman", alternative="greater", exact=FALSE)
cat("Spearman correlation with age on complete dataset:", correlation$estimate, "with P-value", correlation$p.value, "\n\n")
for (i in seq(1:2)) {
    tempdata <- subset(averages.gender, Gender==genders[i])
    correlation <- cor.test(tempdata$Age, tempdata$Sig1Med, method="spearman", alternative="two.sided", exact=FALSE)
    cat("Spearman rho correlation with age on", genders[i], "dataset:", correlation$estimate, "with P-value", correlation$p.value, "\n")
    }

In [ ]:
# Wilcoxon rank sum test
newdata.male <- subset(averages.gender, Gender=='MALE')
newdata.female <- subset(averages.gender, Gender=='FEMALE')
cat(median(newdata.male$Sig1Med/newdata.male$Age)/median(newdata.female$Sig1Med/newdata.female$Age),"\n")
pwres <- wilcox.test(newdata.male$Sig1Med/newdata.male$Age, newdata.female$Sig1Med/newdata.female$Age, exact=FALSE, pairwise=FALSE)
cat("P-value for Male vs Female:", pwres$p.value, '\n')

In [ ]:
# Wilcoxon rank sum test by cohorts
for (i in seq(2)) {
    newdata.male <- subset(data, Gender=='MALE' & Cohort==cohorts[i])
    newdata.female <- subset(data, Gender=='FEMALE' & Cohort==cohorts[i])
    pwres <- wilcox.test(newdata.male$Sig1Total/newdata.male$Age, newdata.female$Sig1Total/newdata.female$Age, alternative="two.sided", exact=FALSE, pairwise=FALSE)
    cat("P-value for Male vs Female in", cohorts[i], ":", pwres$p.value, "\n")
    }

In [ ]:
q <- quantile(data$Sig7Total/data$Age,0.1)
LowSig7data <- subset(data, Sig7Total/Age<q)
nrow(LowSig7data)

In [ ]:
nrow(subset(LowSig7data, Gender=='MALE'))

In [ ]:
# Compute averages in separate data frame (MALE)
agemale <- sort(unique(subset(LowSig7data,Gender=="MALE")$Age)[!is.na(unique(subset(LowSig7data,Gender=="MALE")$Age))])
averages.male.age <- data.frame(Age=agemale)
averages.male.age$Sig1Med <- NA
for (s in agemale) {
  i <- match(s,averages.male.age$Age)
  averages.male.age[i,c("Sig1Med")] <- median(LowSig7data[which(LowSig7data$agemale==s & LowSig7data$Gender=="MALE"),"Sig1Total"])
  averages.male.age[i,c("Sig7Med")] <- median(LowSig7data[which(LowSig7data$agemale==s & LowSig7data$Gender=="MALE"),"Sig7Total"])
}
# Compute averages in separate data frame (FEMALE)
agefemale <- sort(unique(subset(LowSig7data,Gender=="FEMALE")$Age)[!is.na(unique(subset(LowSig7data,Gender=="FEMALE")$Age))])
averages.female.age <- data.frame(Age=agefemale)
averages.female.age$Sig1Med <- NA
for (s in agefemale) {
  i <- match(s,averages.female.age$Age)
  averages.female.age[i,c("Sig1Med")] <- median(LowSig7data[which(LowSig7data$agefemale==s & LowSig7data$Gender=="FEMALE"),"Sig1Total"])
  averages.female.age[i,c("Sig7Med")] <- median(LowSig7data[which(LowSig7data$agefemale==s & LowSig7data$Gender=="FEMALE"),"Sig7Total"])
}
averages.male.age$Gender <- "MALE"
averages.female.age$Gender <- "FEMALE"
averages.gender.age.lowsig7 <- rbind(averages.male.age, averages.female.age)

Estimate the $\alpha$ parameter for male and female.

In [ ]:
res.male <- glm.nb(as.integer(Sig1Med) ~ Age, data=subset(averages.gender, Gender=='MALE' & Age>29))
alpha.male <- coef(res.male)["Age"]
cat(alpha.male, 1/alpha.male, "\n")
res.female <- glm.nb(as.integer(Sig1Med) ~ Age, data=subset(averages.gender, Gender=='FEMALE' & Age>29))
alpha.female <- coef(res.female)["Age"]
cat(alpha.female, 1/alpha.female, "\n")

In [ ]:
summary(res.male)
summary(res.female)

In [ ]:
expfun.male <- function(x) {
    return(unname(exp(coef(res.male)["(Intercept)"]))*exp(unname(coef(res.male)["Age"])*x)/x)
    }
expfun.female <- function(x) {
    return(exp(unname(coef(res.female)["(Intercept)"]))*exp(unname(coef(res.female)["Age"])*x)/x)
    }

In [ ]:
line.male <- data.frame(x = seq(30,90, by=0.1), y=sapply(seq(30,90, by=0.1), expfun.male))
line.female <- data.frame(x = seq(30,90, by=0.1), y=sapply(seq(30,90, by=0.1), expfun.female))

In [ ]:
plot4C <- ggplot(subset(averages.gender, Age>29), aes(Age, Sig1Med, color=Gender)) + 
    geom_smooth(method = "glm", aes(Age, as.integer(Sig1Med)), formula = y ~ x, data=subset(averages.gender, Age>29), se=FALSE, method.args = (family="poisson")) + 
    geom_point() + xlab("Age") + ylab("Signature 1")  + scale_color_aaas() +
    theme_minimal() + theme(legend.position="top") + coord_cartesian(ylim = c(0,200))
#+ scale_color_manual(breaks = c("MALE", "FEMALE"), values=c("#999999", "#E69F00"))
plot4D <- ggplot(subset(averages.gender, Age>29), aes(Age, Sig1Mean/Age, color=Gender)) + geom_point() +
    xlab("Age") + ylab("Signature 1 / Age") + scale_color_aaas() + 
    theme_minimal() + theme(legend.position="none") + coord_cartesian(ylim = c(0,3))+ 
    geom_line(data=line.male, aes(x, y), color='#EE0000FF', size=1) + 
    geom_line(data=line.female, aes(x, y), color='#3B4992FF', size=1)
grid.arrange(plot4C, plot4D, ncol=2, nrow=1)
#p <- arrangeGrob(plot1, plot2, ncol=2, nrow=1)

In [ ]:
mypal = pal_aaas("default", alpha=1)(9)
mypal


In [ ]:
#ggsave("./figures/Figure4CD-exp.pdf", plot=p, scale=1, width=8, height=4)

In [ ]:
plot1 <- ggplot(subset(averages.gender, Age>29), aes(Age, Sig1Med, color=Gender)) + 
    geom_smooth(method = "rlm", aes(Age, Sig1Med), formula = y ~ x, data=subset(averages.gender, Age>29), se=FALSE) + 
    geom_point() + xlab("Age") + ylab("Signature 1")  + scale_color_aaas() +
    theme_minimal() + 
    theme(legend.position="top") + coord_cartesian(ylim = c(0,200))
plot2 <- ggplot(subset(averages.gender, Age>29), aes(Age, Sig1Mean/Age, color=Gender)) + geom_point() +
    xlab("Age") + ylab("Signature 1 / Age")  + scale_color_aaas() +
    theme_minimal() + 
    theme(legend.position="none") + coord_cartesian(ylim = c(0,3))+ 
    geom_smooth(method = "rlm", aes(Age, Sig1Med/Age), formula = y ~ x, data=subset(averages.gender, Age>29), se=FALSE) 
grid.arrange(plot1, plot2, ncol=2, nrow=1)
p <- arrangeGrob(plot1, plot2, ncol=2, nrow=1)

In [ ]:
#ggsave("./figures/Figure4CD-lin.pdf", plot=p, scale=1, width=8, height=4)

#### Repeat the same thing in the presence of MC1R R alleles

#### Incorporate covariates
First, we use the model above with various covariates to determine which ones are significant. It turns out that all are.

In [ ]:
res.simple <- glm(as.integer(Sig1Med) ~ Age + offset(log(Sig7Med)), data=subset(averages.gender, Gender=='FEMALE' & Sig7Med>0), family="poisson")
summary(res.simple)

In [ ]:
res.male <- glm.nb(as.integer(Sig1Total) ~ Age + Cohort + Gender, data=subset(data, Gender=="MALE"))
res.female <- glm.nb(as.integer(Sig1Total) ~ Age + Cohort + Gender, data=subset(data, Gender=="FEMALE"))

Study whether MC1R status is relevant. 

In [ ]:
res.simple <- glm(as.integer(Sig1Total) ~ Age + Gender + Cohort + Rallele, data=mydata, family="poisson")
factor.allele <- exp(coef(res.simple)["Rallele1-2 R alleles"])
cat("The precense of 1-2 MC1R alleles contributes to a multiplicative increase factor of", factor.allele, "across all subtypes.")

In [ ]:
res.simple <- glm(as.integer(Sig1Total) ~ Age + Gender + Cohort + Rallele + Sig7Total, data=mydata, family="poisson")
factor.allele <- exp(coef(res.simple)["Rallele1-2 MC1R alleles"])
cat("The precense of 1-2 MC1R alleles contributes to a multiplicative increase factor of", factor.allele, "across BRAF subtypes.\n")

In [ ]:
res.exp.male <- glm(as.integer(Sig1Total) ~ Age + Sig7Total + Cohort + Site, data = subset(data, Gender=="MALE"), family="poisson")
res.exp.female <- glm(as.integer(Sig1Total) ~ Age + Sig7Total + Cohort + Site, data = subset(data, Gender=="FEMALE"), family="poisson")
N0.male <- exp(coef(res.exp.male)["(Intercept)"])
alpha.male <- coef(res.exp.male)["Age"]
N0.female <- exp(coef(res.exp.female)["(Intercept)"])
alpha.female <- coef(res.exp.female)["Age"]
Sig7.male <- coef(res.exp.male)["Sig7Total"]
Sig7.female <- coef(res.exp.female)["Sig7Total"]
cat(N0.male, "\n")
cat(alpha.male, "\n")
cat(N0.female, "\n")
cat(alpha.female, "\n")
cat(Sig7.male, "\n")
cat(Sig7.female, "\n")

In [ ]:
ggsave("./figures/Figure1A.pdf", plot=plotA, scale=1, width=4, height=4)
ggsave("./figures/Figure1B.pdf", plot=plotB, scale=1, width=4, height=4)
ggsave("./figures/Figure1C.pdf", plot=plotC, scale=1, width=4, height=4)
ggsave("./figures/Figure1D.pdf", plot=plotD, scale=1, width=4, height=4)
ggsave("./figures/Figure1E.pdf", plot=plotE, scale=1, width=4, height=4)
ggsave("./figures/Figure1F.pdf", plot=plotF, scale=1, width=4, height=4)
ggsave("./figures/Figure2A.pdf", plot=plot2A, scale=1, width=4, height=4)
ggsave("./figures/Figure2B.pdf", plot=plot2B, scale=1, width=4, height=4)
ggsave("./figures/Figure2C.pdf", plot=plot2C, scale=1, width=4, height=4)
ggsave("./figures/Figure2D.pdf", plot=plot2D, scale=1, width=4, height=4)
ggsave("./figures/Figure2E.pdf", plot=plot2E, scale=1, width=8, height=4)
ggsave("./figures/Figure3A.pdf", plot=plot3A, scale=1, width=4, height=4)
ggsave("./figures/Figure3B.pdf", plot=plot3B, scale=1, width=4, height=4)
ggsave("./figures/Figure3C.pdf", plot=plot3C, scale=1, width=4, height=4)
ggsave("./figures/Figure4A.pdf", plot=plot4A, scale=1, width=4, height=4)
ggsave("./figures/Figure4B.pdf", plot=plot4B, scale=1, width=4, height=4)
ggsave("./figures/Figure4C.pdf", plot=plot4C, scale=1, width=4, height=4)
ggsave("./figures/Figure4D.pdf", plot=plot4D, scale=1, width=4, height=4)